In [86]:
import gspread
import time
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

doctor_list = []

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(credentials)

sheet_tarih = client.open("Shift_optimization").worksheet('1. Tarih')
sheet_takvim_input = client.open("Shift_optimization").worksheet('2. Takvim')
sheet_birim_dagilimi = client.open("Shift_optimization").worksheet('3. Birim dagilimi')

sheet_tarih_data = sheet_tarih.get_all_records()
sheet_takvim_input_data = sheet_takvim_input.get_all_records()
sheet_birim_dagilimi_data = sheet_birim_dagilimi.get_all_records()

nobetci_sayisi = int(sheet_birim_dagilimi.cell(1,2).value)  

class doctor(object):
    name = ""
    rank = 0
    adjusted_rank = 0
    shift = False
    nb_of_shifts = 0
    weekend = False
    nb_of_weekends = 0
    days_off = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    nb_days_off = 0
    #inme calendar eklenicek
    manual_shifts = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    wanted_days = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    unwanted_days = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    shift_calendar = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    # The class "constructor" - It's actually an initializer 
    def __init__(self, name, rank, adjusted_rank, shift, nb_of_shifts, weekend, nb_of_weekends, days_off, nb_days_off, manual_shifts, wanted_days, unwanted_days, shift_calendar):
        self.name = name
        self.rank = rank
        self.adjusted_rank = adjusted_rank
        self.shift = shift
        self.nb_of_shifts = nb_of_shifts
        self.weekend = weekend
        self.nb_of_weekends = nb_of_weekends
        self.days_off = days_off
        self.nb_days_off = nb_days_off
        self.manual_shifts = manual_shifts
        self.wanted_days = wanted_days 
        self.unwanted_days = unwanted_days
        self.shift_calendar = shift_calendar
        
        
class month(object):
    days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
    weekend = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    special_shift = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]    

    # The class "constructor" - It's actually an initializer 
    def __init__(self, days, weekend, special_shift):
        self.days = days
        self.weekend = weekend
        self.special_shift = special_shift


def GetNurseInfo(nobetci_sayisi):
    
    num_days = int(sheet_tarih.cell(9,2).value)+int(sheet_tarih.cell(10,2).value)
    print("Number of days : " + str(num_days))
    total_rank = 0
    col = []
    
    name = ""
    rank = 0
    adjusted_rank = 0
    shift = False
    nb_of_shifts = 0
    weekend = False
    nb_of_weekends = 0
    nb_days_off = 0
    
    num_shifts = 1 #number of shifts per day
    total_shift_rounded = 0
    
    for n in range(nobetci_sayisi):
        days_off = []
        manual_shifts = []
        wanted_days = []
        unwanted_days = []
        shift_calendar = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        col.append(sheet_takvim_input.col_values(n+4))
        name = col[n][1]
        rank = col[n][2]
        shift = col[n][3]
        weekend =col[n][4]
        nb_days_off = col[n][37]
        adjusted_rank = int(rank)*(num_days-int(nb_days_off))/num_days
        
       
        if shift == "Evet":
            shift = True
        else:
            shift = False
               
        if weekend == "Evet":
            weekend = True
        else:
            weekend = False
                        
        for l in range(num_days):
            if col[n][5+l] == "Izinli":
                days_off.append(1)
                manual_shifts.append(0)
                wanted_days.append(0)
                unwanted_days.append(0)
            elif col[n][5+l] == "Nöbet":
                days_off.append(0)
                manual_shifts.append(1)
                wanted_days.append(0)
                unwanted_days.append(0)
            elif col[n][5+l] == "Tercih":
                days_off.append(0)
                manual_shifts.append(0)
                wanted_days.append(1)
                unwanted_days.append(0)
            elif col[n][5+l] == "Istemiyor":
                days_off.append(0)
                manual_shifts.append(0)
                wanted_days.append(0)
                unwanted_days.append(1)
            elif col[n][5+l] == "":
                days_off.append(0)
                manual_shifts.append(0)
                wanted_days.append(0)
                unwanted_days.append(0)
 
        doctor_list.append(doctor(name, rank, adjusted_rank, shift, nb_of_shifts, weekend, nb_of_weekends, days_off, nb_days_off, manual_shifts, wanted_days, unwanted_days, shift_calendar))
  
    for v in range(nobetci_sayisi):
        total_rank = total_rank + doctor_list[v].adjusted_rank
        #print("Adjusted rank for doctor " + str(v+1) + " = " + str(doctor_list[v].adjusted_rank))
        
    total_rank = round(total_rank)
    
    number_of_credits = int(sheet_tarih.cell(14,3).value)
    number_of_weekends = int(sheet_tarih.cell(10,2).value)
    
    for n in range(nobetci_sayisi):
        #if doctor_list[n].weekend = False:
         #   doctor_list[n].nb_of_weekends = 0
        #else:
         #   doctor_list[n].nb_of_weekends = round((number_of_weekends/total_rank)*doctor_list[n].adjusted_rank) 
        
        
        
        
        doctor_list[n].nb_of_shifts = round((num_shifts*num_days/total_rank)*doctor_list[n].adjusted_rank)        
        total_shift_rounded = total_shift_rounded + doctor_list[n].nb_of_shifts
        
        print("Name : " + doctor_list[n].name)
        print("Shift numbers : " + str(doctor_list[n].nb_of_shifts))

    
    shift_difference = num_shifts*num_days - total_shift_rounded
    
    print("Total shifts rounded : " + str(total_shift_rounded))
    
    if shift_difference == 3:
        print("Shift difference 3")
        doctor_list[n].nb_of_shifts = doctor_list[n].nb_of_shifts+2
        doctor_list[n-1].nb_of_shifts = doctor_list[n-1].nb_of_shifts+1
        
    elif shift_difference == 2:
        print("Shift difference 2")
        doctor_list[n].nb_of_shifts = doctor_list[n].nb_of_shifts+1
        doctor_list[n-1].nb_of_shifts = doctor_list[n-1].nb_of_shifts+1
        
    elif shift_difference == 1:
        print("Shift difference 1")
        doctor_list[n].nb_of_shifts = doctor_list[n].nb_of_shifts+1
    
    
    if shift_difference > 0:
        print("Shift difference positive : " + str(shift_difference))
        distribution_counter = shift_difference
        print("Distribution counter = " + str(distribution_counter))
        j = 1
        while distribution_counter > 0:
            print("Actual shifts for doctor" + str(nobetci_sayisi) +" : " + str(doctor_list[nobetci_sayisi-j].nb_of_shifts))
            print("Looking at Doctor " + str(nobetci_sayisi-j+1))
            doctor_list[nobetci_sayisi-j].nb_of_shifts = doctor_list[nobetci_sayisi-j].nb_of_shifts + 1
            print("New shift number for doctor " + str(nobetci_sayisi-j+1) + " : " + str(doctor_list[nobetci_sayisi-j].nb_of_shifts))
            distribution_counter = distribution_counter - 1
                
            j = j+1    
    
    
    elif shift_difference < 0:
        print("Shift difference negative : " + str(shift_difference))
        distribution_counter = shift_difference
        print("Distribution counter = " + str(distribution_counter))
        k = 0
        while distribution_counter < 0:
            print("Actual shifts for doctor" + str(k+1) +" : " + str(doctor_list[k].nb_of_shifts))
            if doctor_list[k].nb_of_shifts > 0: 
                print("Looking at Doctor " + str(k+1))
                doctor_list[k].nb_of_shifts = doctor_list[k].nb_of_shifts - 1
                print("New shift number for doctor " + str(k+1) + " : " + str(doctor_list[k].nb_of_shifts))
                distribution_counter = distribution_counter + 1
                
            k = k+1
        

    #for n in range(nobetci_sayisi):
        #print("Total shift for doctor number" + str(n+1) + " : " + str(doctor_list[n].nb_of_shifts))



In [91]:
#Calculate shift distribution
GetNurseInfo(nobetci_sayisi)

Number of days : 31
Total rank = 23
Name : Dinç
Shift numbers : 1
Name : Mert
Shift numbers : 2
Name : Ceren
Shift numbers : 3
Name : Ali
Shift numbers : 3
Name : Emre
Shift numbers : 4
Name : Hüseyin
Shift numbers : 5
Name : Ezgi
Shift numbers : 7
Name : Elif
Shift numbers : 7
Shift difference negative : -1
Distribution counter = -1
Actual shifts for doctor1 : 1
Looking at Doctor 1
New shift number for doctor 1 : 0


In [ ]:
#pprint(sheet_takvim_input_data)

In [ ]:
#print all details for each doctor
for n in range(nobetci_sayisi):
    print("------ Doctor number " + str(n+1) + " ------")
    print("Name : " + doctor_list[n].name)
    print("Rank : " + doctor_list[n].rank)
    print("Adjusted rank : "+ str(doctor_list[n].adjusted_rank))
    print("Does shifts ? " + str(doctor_list[n].shift))
    print("Number of shifts : " + str(doctor_list[n].nb_of_shifts))
    print("Does weekends ? " + str(doctor_list[n].weekend))
    print("Days off : " + str(doctor_list[n].days_off))
    print("Total days off : " + str(doctor_list[n].nb_days_off))
    print("Manually appointed shifts : " + str(doctor_list[n].manual_shifts))
    print("Wanted shift days : " + str(doctor_list[n].wanted_days))
    print("Unwanted shift days : " + str(doctor_list[n].unwanted_days))
    print("Shift calendar : " + str(doctor_list[n].shift_calendar))

In [90]:
#insert values to excel tab "3. Birim Dagilimi"

nameRange = "A9:A" + str(nobetci_sayisi+8)
shiftRange = "B9:B" + str(nobetci_sayisi+8)

cellList = sheet_birim_dagilimi.range(nameRange)
for cell in cellList:
    cell.value = doctor_list[cell.row-9].name
sheet_birim_dagilimi.update_cells(cellList)

shiftCellList = sheet_birim_dagilimi.range(shiftRange)
for cell in shiftCellList:
    cell.value = doctor_list[cell.row-9].nb_of_shifts
sheet_birim_dagilimi.update_cells(shiftCellList)
    

{'spreadsheetId': '1bdXz3_N18NCq1F3Zk1kiqDSAH_-dRc87yXG5IZCFtLc',
 'updatedRange': "'3. Birim dagilimi'!B9:B16",
 'updatedRows': 8,
 'updatedColumns': 1,
 'updatedCells': 8}